<a href="https://colab.research.google.com/github/kiranavhad87/Python/blob/master/Sentiment%20analysis%20using%20BoW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../datasets/Restaurant_Reviews.tsv',sep='\t')

In [ ]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
df.shape

(1000, 2)

In [ ]:
msg = 'Wow... Loved this place.'

In [ ]:
# 1. Lower case everything
msg = msg.lower()
msg

'wow... loved this place.'

In [ ]:
# 2. remove unwanted chars.... (Keep Alphabets)
import re  # regular expression

In [ ]:
msg = re.sub('[^a-z]',' ',msg)

In [ ]:
msg

'wow    loved this place '

In [ ]:
# 3.Remove Stopwords => nltp ==> !pip install nltk
import nltk

In [ ]:
nltk.corpus.stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
msg

'wow    loved this place '

In [ ]:
list_of_words = msg.split()

In [ ]:
list_of_words

['wow', 'loved', 'this', 'place']

In [ ]:
list_without_stopwords = []
for word in list_of_words:
    if word in nltk.corpus.stopwords.words('english'):
        print(word, 'is stopword')
    else:
        list_without_stopwords.append(word)

this is stopword


In [ ]:
list_without_stopwords = [word for word in list_of_words if not word in nltk.corpus.stopwords.words('english')] 

In [ ]:
list_without_stopwords

['wow', 'loved', 'place']

In [ ]:
# Stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
ps.stem('loving')

'love'

In [ ]:
list_of_stemmed_words = [ps.stem(word) for word in list_without_stopwords]

In [ ]:
list_of_stemmed_words

['wow', 'love', 'place']

In [ ]:
msg = ' '.join(list_of_stemmed_words)
msg

'wow love place'

In [ ]:
def get_processed(msg):
    msg = re.sub('[^a-zA-Z]',' ',msg)
    msg = msg.lower()
    list_A = msg.split()
    list_B = [ps.stem(word) for word in list_A if not word in nltk.corpus.stopwords.words('english')]
    msg = ' '.join(list_B)
    return msg

In [ ]:
get_processed('Wow... Loved this place.')

'wow love place'

In [ ]:
df['Review'].apply(get_processed)

0                                         wow love place
1                                             crust good
2                                     tasti textur nasti
3      stop late may bank holiday rick steve recommen...
4                                select menu great price
                             ...                        
995                        think food flavor textur lack
996                               appetit instantli gone
997                         overal impress would go back
998    whole experi underwhelm think go ninja sushi n...
999    wast enough life pour salt wound draw time too...
Name: Review, Length: 1000, dtype: object

In [ ]:
# Build BoW model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
BoW = CountVectorizer()
TFIDF_table = TfidfVectorizer()
BoW_table = BoW.fit_transform(df['Review'].apply(get_processed))


In [ ]:
BoW_table

<1000x1565 sparse matrix of type '<class 'numpy.int64'>'
	with 5372 stored elements in Compressed Sparse Row format>

In [ ]:
X = BoW_table.toarray()

In [ ]:
X.shape

(1000, 1565)

In [ ]:
sum(X[0])

3

In [ ]:
y = df['Liked']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(random_state=56)
model.fit(X,y)

C:\Users\Vivek\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=56, verbose=0,
                       warm_start=False)

In [ ]:
model.score(X,y)

0.98

In [ ]:
new_review = 'Very nearer to swargate and good service'

In [ ]:
temp = get_processed(new_review)

In [ ]:
a1 = BoW.transform([temp])
a1 = a1.toarray()
a1

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
model.predict(a1)[0]

1

In [ ]:
def get_sentiment(review):
    review = get_processed(review)
    a1 = BoW.transform([review])
    a1 = a1.toarray()
    return model.predict(a1)[0]

In [ ]:
get_sentiment('service was not good')

1

In [ ]:
get_processed('service was good')

'servic good'

In [ ]:
get_processed('service was not good')

'servic good'